In [27]:
!pip install yfinance pandas lxml beautifulsoup4 plotly requests


In [28]:
import yfinance as yf
import pandas as pd
from bs4 import BeautifulSoup
import requests


In [29]:
# Download Tesla stock data
tesla_data = yf.Ticker("TSLA").history(period="max")

# Reset the index
tesla_data.reset_index(inplace=True)

# Display first 5 rows
tesla_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [31]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Tesla revenue URL
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

# Add headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 " +
                  "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
}

# Request the page
response = requests.get(url, headers=headers)

# Parse HTML
soup = BeautifulSoup(response.text, "html.parser")

# Find all tables
tables = soup.find_all("table")

# The revenue table usually has 'Tesla Quarterly Revenue' in it
for table in tables:
    if "Tesla Quarterly Revenue" in table.text:
        revenue_table = table
        break

# Extract rows
rows = revenue_table.find_all("tr")

# Create DataFrame
tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])
for row in rows[1:]:  # skip header
    cols = row.find_all("td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace("$","").replace(",","")
        tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame({"Date":[date], "Revenue":[revenue]})], ignore_index=True)

# Keep only non-empty revenues
tesla_revenue = tesla_revenue[tesla_revenue["Revenue"] != ""]

# Reset index
tesla_revenue.reset_index(drop=True, inplace=True)

# Show last 5 rows
tesla_revenue.tail()


,Date,Revenue
59,2010-09-30,31
60,2010-06-30,28
61,2010-03-31,21
62,2009-09-30,46
63,2009-06-30,27


In [32]:
# Download GameStop stock data
gme_data = yf.Ticker("GME").history(period="max")

# Reset the index
gme_data.reset_index(inplace=True)

# Display the first 5 rows
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693349,1.603295,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658001,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


In [34]:
# URL for GameStop revenue
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

# Set headers to mimic a real browser
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                         "AppleWebKit/537.36 (KHTML, like Gecko) "
                         "Chrome/115.0.0.0 Safari/537.36"}

# Get page content
response = requests.get(url, headers=headers)
html_data = response.text

# Parse tables using pandas from the HTML content
tables = pd.read_html(html_data)

# Filter the revenue table (usually the second table)
gme_revenue = tables[1]

# Reset index
gme_revenue.reset_index(drop=True, inplace=True)

# Display last 5 rows
gme_revenue.tail()

,GameStop Quarterly Revenue (Millions of US $),GameStop Quarterly Revenue (Millions of US $).1
61,2010-01-31,"$3,524"
62,2009-10-31,"$1,835"
63,2009-07-31,"$1,739"
64,2009-04-30,"$1,981"
65,2009-01-31,"$3,492"


In [40]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [41]:
def make_graph(stock_data, revenue_data, stock_name="Tesla"):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        subplot_titles=(f"{stock_name} Stock Price", f"{stock_name} Revenue"),
                        vertical_spacing = 0.3)
    
    # Stock price trace
    fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['Close'], name="Stock Price"), row=1, col=1)
    
    # Revenue trace
    fig.add_trace(go.Bar(x=revenue_data['Date'], y=revenue_data['Revenue'], name="Revenue"), row=2, col=1)
    
    # Layout update
    fig.update_layout(showlegend=False, height=600, width=900,
                      title_text=f"{stock_name} Stock Price and Revenue")
    
    fig.show()


In [42]:
make_graph(tesla_data, tesla_revenue, stock_name="Tesla")


In [73]:
def make_graph(stock_data, stock_name="GameStop"):
    fig = go.Figure()
    
    # Stock price trace
    fig.add_trace(go.Scatter(
        x=stock_data['Date'],
        y=stock_data['Close'],
        mode='lines',
        name=f"{stock_name} Stock Price"
    ))
    
    # Layout
    fig.update_layout(
        title=f"{stock_name} Stock Price",
        xaxis_title="Date",
        yaxis_title="Stock Price (USD)",
        height=500,
        width=900
    )
    
    fig.show()


In [74]:
make_graph(gme_data, stock_name="GameStop")
